In [1]:
# Mark as true if you have the courseRecordDict.json file in the directory.
# If not, mark False, will generate for future ease (fewer api calls)
courseRecordDictExists = False

# Mark as true if you have the userInfoDict.json file in the directory.
# If not, mark False, will generate for future ease (fewer api calls)
userInfoDictExists = False

# Need to update this every time we run (new batch). 
# Gotta do this by trial error, no other way
maxUserId = 18000

In [2]:
import json
from bpApi import *

# Collect data from api

In [3]:
if userInfoDictExists:
    with open('userInfoDict.json', 'r') as fp:
        userInfoDict = json.load(fp)
else:
    userInfoDict = {}

In [33]:
if userInfoDictExists:
    maxTried = max(map(lambda x : int(x), userInfoDict.keys()))
else:
    maxTried = 0

In [35]:
for userID in range(0, maxTried):
    
    if userID in userInfoDict and 'iiitd.ac.in' in userInfoDict[userID][1]:
        response = getUserInfoAndCourses(userID)
    else:
        response = False

    if response == False:
        pass
    else:
        print userID, response
        userInfoDict[userID] = response

    if userID % 100 == 0:
        print 'Writing to file'
        with open('userInfoDict.json', 'w') as fp:
            json.dump(userInfoDict, fp)
            
with open('userInfoDict.json', 'w') as fp:
    json.dump(userInfoDict, fp)

In [26]:
for userID in range(maxTried, maxUserId):
    
    response = getUserInfoAndCourses(userID)
    
    if response == False:
        pass
    else:
        print userID, response
        userInfoDict[userID] = response

    if userID % 1000 == 0:
        print 'Writing to file'
        with open('userInfoDict.json', 'w') as fp:
            json.dump(userInfoDict, fp)
            
with open('userInfoDict.json', 'w') as fp:
    json.dump(userInfoDict, fp)

# Load data

In [36]:
with open('userInfoDict.json', 'r') as fp:
    userInfoDict = json.load(fp)
print max(map(lambda x : int(x), userInfoDict.keys())), len(userInfoDict)

17883 12474


In [37]:
iiitdCount = 0
for userKey in userInfoDict.keys():
    if 'iiitd.ac.in' in userInfoDict[userKey][1]:
        iiitdCount += 1
print "IIITD Users :", iiitdCount
print "Total Users :", len(userInfoDict)

IIITD Users : 2646
Total Users : 12474


# Collect records for courses taken by IIITD students

In [38]:
if courseRecordDictExists:
    with open('courseRecordDict.json', 'r') as fp:
        courseRecordDict = json.load(fp)
else:
    courseRecordDict = {}

In [85]:
for userKey in userInfoDict.keys():
    userEmail  = userInfoDict[userKey][1]
    allCourses = userInfoDict[userKey][2]
    
    if 'iiitd.ac.in' in userEmail:
        # Make queries for courses registered by IIITD students only
        print userEmail, allCourses

        for courseID in allCourses:

            if courseID not in courseRecordDict:
                # Check if fetched before
                print "Making Query for course ID", courseID, '\n'
                courseInfo = getCourseInfo(courseID)
                courseRecordDict[str(courseID)] = (courseInfo['code'], courseInfo['name'])
                # print "Added course ID", courseID, courseRecordDict[str(courseID)]
            else:
                # print "Found course ID", courseID, courseRecordDict[str(courseID)]
                pass

In [45]:
with open('courseRecordDict.json', 'w') as fp:
    json.dump(courseRecordDict, fp)

In [46]:
courseRecords = {}

for _id in courseRecordDict.keys():
    desc = list(courseRecordDict[_id])
    courseRecords[_id] = {}
    courseRecords[_id]['code'] = desc[0]
    courseRecords[_id]['name'] = desc[1]
    courseRecords[_id]['students'] = []

In [47]:
allCourseCodes = map(lambda x : x['code'], courseRecords.values())
allCourseNames = map(lambda x : x['name'], courseRecords.values())

print len(allCourseCodes), len(set(allCourseCodes)), len(set(allCourseNames))

617 417 372


# Map students to courses

In [48]:
for userKey in userInfoDict.keys():
    if 'iiitd.ac.in' in userInfoDict[userKey][1]:
        allCourses = userInfoDict[userKey][2]
        for course in allCourses:
            # print courseRecordDict[course], userInfoDict[userKey]
            courseRecords[course]['students'].append(userInfoDict[userKey])

In [84]:
for _id in courseRecords.keys():
    print courseRecords[_id]['code'], courseRecords[_id]['name'], len(courseRecords[_id]['students'])

In [50]:
with open('courseRecords.json', 'w') as fp:
    json.dump(courseRecords, fp)

# Merge courses based on names. In case course id was changed over time

In [71]:
finalCourseList = {}
cleanFinalCourseList = {}

In [72]:
for _id in courseRecords.keys():
    name = courseRecords[_id]['name']
    code = courseRecords[_id]['code']
    
    if name not in finalCourseList:
        finalCourseList[name] = {'codes' : [],
                                 'students' : []}
    
    finalCourseList[name]['codes'].append(code)
    finalCourseList[name]['students'] += courseRecords[_id]['students']

In [73]:
for courseName in finalCourseList.keys():
    for i in range(len(finalCourseList[courseName]['students'])):
        finalCourseList[courseName]['students'][i] = finalCourseList[courseName]['students'][i][:-1]

In [74]:
for key in finalCourseList:
    
    uniqueStudents = [list(x) for x in set(tuple(x) for x in finalCourseList[key]['students'])]
    finalCourseList[key]['students'] = uniqueStudents


In [75]:
with open('finalCourseList.json', 'w') as fp:
    json.dump(finalCourseList, fp)

In [81]:
len(finalCourseList)

372

In [83]:
# finalCourseList[finalCourseList.keys()[0]]